In [12]:
import pandas as pd
from tensorflow.keras.models import load_model
import pickle
import numpy as np

In [4]:
model=load_model('model.keras')

with open('gender_labelencoder.pkl','rb') as file:
    gender_labelencoder=pickle.load(file)

with open('geo_onehotencoder.pkl','rb') as file:
    geo_onehotencoder=pickle.load(file)

with open('standardscaler.pkl','rb') as file:
    standardscaler=pickle.load(file)

C:\Users\91781\AppData\Roaming\Python\Python311\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [5]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [34]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [37]:
geo_encoded=geo_onehotencoder.transform([input_df['Geography']]).toarray()
geo_encoded=pd.DataFrame(geo_encoded,columns=geo_onehotencoder.get_feature_names_out(['Geography']))
geo_encoded

c:\Users\91781\OneDrive\Desktop\MLProjects\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [40]:
input_df['Gender']=gender_labelencoder.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [42]:
input_df=pd.concat([input_df.drop(['Geography'],axis=1),geo_encoded],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [44]:
input_scaled=standardscaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [45]:
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 495ms/step


array([[0.0482407]], dtype=float32)

In [49]:
prob=prediction[0][0]

In [50]:
if prob>0.5:
    print('Customer Likely to Churn')
else:
    print('Customer not likely to churn')

Customer not likely to churn
